In [1]:
import numpy as np
import pandas as pd
from random import randint,sample

import warnings
warnings.filterwarnings('ignore')

In [2]:
## cloning the repositry from the github to make the file easily shareable
!git clone https://github.com/guru-narayana/NNFL-assignment

fatal: destination path 'NNFL-assignment' already exists and is not an empty directory.


In [3]:
## loading the data
data_frame = pd.read_excel("/content/NNFL-assignment/data5.xlsx").sample(frac = 1)
data = data_frame.to_numpy()

## seperating input and output data
input_prenorm_data = data[:,0:data.shape[1]-1]
op = data[:,data.shape[1]-1]

## normalising the data
max_data  =  np.amax(input_prenorm_data,axis=0) ## max  values in each column
input_data  =   input_prenorm_data/max_data
output_data = np.zeros((len(op),3))

#Auto encoding
for i in range(len(op)):
  output_data[i,int(op[i]-1)] = 1

In [4]:
def gaussian_activation(a,b,x):
  P = np.zeros((x.shape[0],a.shape[0]))
  for i in range(a.shape[0]):
    P[:,i] = -b[i]*np.sqrt(np.sum((x-a[i])**2,axis=1))
  return np.exp(P)


def tanh_activation(a,b,x):
  V = -(x@a.T + b)
  P = (1 - np.exp(V))/(1 + np.exp(V))
  return P

def confiusion_matrix_func(outpt,Y):
  pred = np.argmax(outpt, axis=1)+1
  output_test_data = np.argmax(Y, axis=1)+1
  confusion_matrix = np.zeros((3,3))
  count = 0
  for x in range(len(pred)):
    if(pred[x] == output_test_data[x]):
      count+=1
    if(pred[x] == 1):
      if(output_test_data[x] == 1):
        confusion_matrix[0][0]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][0]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][0]+=1
    elif(pred[x] == 2):
      if(output_test_data[x]   == 1):
        confusion_matrix[0][1]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][1]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][1]+=1
    elif(pred[x] == 3):
      if(output_test_data[x]   == 1):
        confusion_matrix[0][2]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][2]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][2]+=1
  c1 = confusion_matrix[0][0]/np.sum(confusion_matrix[0][:])
  c2 = confusion_matrix[1][1]/np.sum(confusion_matrix[1][:])
  c3 = confusion_matrix[2][2]/np.sum(confusion_matrix[2][:])
  A  = count/len(pred)
  return [c1,c2,c3,A]

In [5]:
def k_fold(hidden_layer_size,model):
  K_size = int(input_data.shape[0]/5)
  split = [0,K_size,2*K_size,3*K_size,4*K_size,input_data.shape[0]-1]
  C1 = 0
  C2 = 0
  C3 = 0
  A  = 0
  for i in range(5):
    input_train_data = np.concatenate((input_data[:split[i]],input_data[split[i+1]:]),axis=0 )
    input_test_data = input_data[split[i]:split[i+1]]
    output_train_data = np.concatenate((output_data[:split[i]],output_data[split[i+1]:]),axis=0 )
    output_test_data = output_data[split[i]:split[i+1]]
    a =  np.random.rand(hidden_layer_size,input_train_data.shape[1])
    b =  np.random.rand(hidden_layer_size)
    H = model(a,b,input_train_data)
    H = np.insert(H,0,np.ones(H.shape[0]),1)
    W = np.linalg.pinv(H.T @ H)@ H.T @ output_train_data
    H_test = model(a,b,input_test_data)
    H_test = np.insert(H_test,0,np.ones(H_test.shape[0]),1)
    pred = H_test@W
    c1,c2,c3,a = confiusion_matrix_func(pred,output_test_data)
    C1+=c1
    C2+=c2
    C3+=c3
    A+=a
  print(model.__name__)
  print("class_1  Accuracy  : "+ str( C1/5 ))
  print("class_2  Accuracy  : "+ str( C2/5 ))
  print("class_3  Accuracy  : "+ str( C3/5 ))
  print( "Overall Accuracy  : " +str(  A/5))
  print("\n")


In [6]:
k_fold(100,gaussian_activation)
k_fold(100,tanh_activation)


gaussian_activation
class_1  Accuracy  : 0.8792483660130719
class_2  Accuracy  : 0.9580952380952381
class_3  Accuracy  : 0.9528070175438597
Overall Accuracy  : 0.9271618625277162


tanh_activation
class_1  Accuracy  : 0.8777777777777779
class_2  Accuracy  : 0.9857142857142858
class_3  Accuracy  : 0.9472514619883041
Overall Accuracy  : 0.932039911308204


